In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [46]:
import re
with open('complete_tex_4.tex','r') as f:
  latex_string = f.readlines()

figure_list = []
Section_headings = {}
for count,lines in enumerate(latex_string):
  # print(lines)
  if re.match(r'\\section', lines):
    head = lines.lstrip(r'\\section{').rstrip().rstrip('}')
    Section_headings[head] = 0
  if re.search(r'\\includegraphics', lines):
    try:
      figure_list.append(lines.split(']{')[1].rstrip().rstrip('}'))
    except:
      pass
print(figure_list)
print(Section_headings)


['figures/gutenberg_AUC_curve.png', 'figures/arxiv_AUC_curve.png', 'figures/book_dates_original.png', 'figures/book_dates_filtered.png', 'figures/across_models_books.png', 'figures/token_freq_comparison.png', 'figures/n_tokens.png']
{'Introduction': 0, 'Background': 0, 'Auditing setup': 0, 'Methodology': 0, 'Experimental setup': 0, 'Results': 0, 'Discussion': 0, 'Related work': 0, 'Conclusion': 0}


In [32]:

def prompt_builder(title,page_length,count):
    return f'''
    Title {count}: {title}
    <Important Detailed Information> ({page_length} Slides)
    Output:
    <Figure path (.png/.jpg) if present in {title}>
    Output:
    <Table from Latex to Markdown if present in section>
    Output:
    '''

user_input = input('Enter the page length for each section: ')
user_input = user_input.split(',')
print(user_input)

for count,key in enumerate(Section_headings):
  Section_headings[key] = user_input[count]

prompt_base = ''
for count,(key,val) in enumerate(Section_headings.items()):
  prompt_base+=prompt_builder(key,val,count+1)

print(prompt_base)

Enter the page length for each section: 3,1,1,1
['3', '1', '1', '1']

    Title 1: Introduction
    <Important Detailed Information> (3 Slides)
    Output:
    <Figure path (.png/.jpg) if present in Introduction>
    Output:
    <Table from Latex to Markdown if present in section>
    Output:
    
    Title 2: Method
    <Important Detailed Information> (1 Slides)
    Output:
    <Figure path (.png/.jpg) if present in Method>
    Output:
    <Table from Latex to Markdown if present in section>
    Output:
    
    Title 3: Experiments
    <Important Detailed Information> (1 Slides)
    Output:
    <Figure path (.png/.jpg) if present in Experiments>
    Output:
    <Table from Latex to Markdown if present in section>
    Output:
    
    Title 4: Conclusion
    <Important Detailed Information> (1 Slides)
    Output:
    <Figure path (.png/.jpg) if present in Conclusion>
    Output:
    <Table from Latex to Markdown if present in section>
    Output:
    


In [33]:
messages=[
    {
      "role": "system",
      "content": "You are an expert slide expert that extracts latex papers and create presentation slides",
    },
    {
      "role": "user",
      "content": f'''Given the document: {latex_string}
      Analyse paper by section and Construct output following this template (Each Slide - 3 bullets):
      {prompt_base}
      ''',
    }
  ]
print(messages)

[{'role': 'system', 'content': 'You are an expert slide expert that extracts latex papers and create presentation slides'}, {'role': 'user', 'content': 'Given the document: [\'\\\\documentclass[sigconf,nonacm]{acmart}\\n\', \'\\\\makeatletter\\n\', \'\\\\renewcommand\\\\@formatdoi[1]{\\\\ignorespaces}\\n\', \'\\\\makeatother\\n\', \'\\n\', \'\\n\', \'\\\\AtBeginDocument{\\\\providecommand\\\\BibTeX{{\\\\normalfont B\\\\kern-0.5em{\\\\scshape i\\\\kern-0.25em b}\\\\kern-0.8em\\\\TeX}}}\\n\', \'\\n\', \'\\n\', \'\\\\acmDOI{}\\n\', \'\\\\acmISBN{}\\n\', \'\\n\', \'\\n\', \'\\\\usepackage{hyperref}\\n\', \'\\\\usepackage{siunitx}\\n\', \'\\\\usepackage{xspace}\\n\', \'\\\\usepackage{multirow}\\n\', \'\\\\usepackage{enumitem}\\n\', \'\\\\usepackage{placeins}\\n\', \'\\\\usepackage{natbib}\\n\', \'\\\\newcommand{\\\\ours}{\\\\textsc{TinyLLM}\\\\xspace}\\n\', \'\\\\settopmatter{printacmref=false}\\n\', \'\\\\setcopyright{none}\\n\', \'\\\\pagestyle{plain}\\n\', \'\\n\', \'\\n\', \'\\\\begin{d

In [15]:
# messages=[
#     {
#       "role": "system",
#       "content": "You are an LaTex reviewer",
#     },
#     {
#       "role": "user",
#       "content": f'''Given the document: {latex_string}
#       Read the paper by section, at indicate which section does {figure_list} belong to? Pick from {Section_headings.keys()}
#       ''',
#     }
#   ]
print(messages)

[{'role': 'system', 'content': 'You are an LaTex reviewer'}, {'role': 'user', 'content': 'Given the document: [\'\\\\documentclass[sigconf,nonacm]{acmart}\\n\', \'\\\\makeatletter\\n\', \'\\\\renewcommand\\\\@formatdoi[1]{\\\\ignorespaces}\\n\', \'\\\\makeatother\\n\', \'\\n\', \'\\n\', \'\\\\AtBeginDocument{\\\\providecommand\\\\BibTeX{{\\\\normalfont B\\\\kern-0.5em{\\\\scshape i\\\\kern-0.25em b}\\\\kern-0.8em\\\\TeX}}}\\n\', \'\\n\', \'\\n\', \'\\\\acmDOI{}\\n\', \'\\\\acmISBN{}\\n\', \'\\n\', \'\\n\', \'\\\\usepackage{hyperref}\\n\', \'\\\\usepackage{siunitx}\\n\', \'\\\\usepackage{xspace}\\n\', \'\\\\usepackage{multirow}\\n\', \'\\\\usepackage{enumitem}\\n\', \'\\\\usepackage{placeins}\\n\', \'\\\\usepackage{natbib}\\n\', \'\\\\newcommand{\\\\ours}{\\\\textsc{TinyLLM}\\\\xspace}\\n\', \'\\\\settopmatter{printacmref=false}\\n\', \'\\\\setcopyright{none}\\n\', \'\\\\pagestyle{plain}\\n\', \'\\n\', \'\\n\', \'\\\\begin{document}\\n\', \'\\n\', \'\\n\', \'\\\\title{\\\\ours: Learning

In [36]:
from openai import OpenAI
import os

TOGETHER_API_KEY = ''

client = OpenAI(
  api_key=TOGETHER_API_KEY,
  base_url='https://api.together.xyz/v1',
)

chat_completion = client.chat.completions.create(
  messages=messages,
  model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
)

print(chat_completion.choices[0].message.content)

APIConnectionError: Connection error.

In [ ]:
import json

my_dict = {f'Title':
            {
                'Information':'Some info',
                'Page Number':'Some info',
                'Images':'Some Image',
                'Tables':'Some Table'
                }
}
